In [1]:
import numpy as np
import torch
import torch.nn as nn
from image import *
from kitti_util_tracking import *
from kitti_object_tracking import *
from Tracker import *
from PIL import Image

In [2]:
data_dir = 'G:\\KITTI\\tracking'

In [3]:
kitti_object = kitti_object_tracking(data_dir)

In [4]:
calib_object = kitti_object.get_calibration(0)

In [5]:
label_object = kitti_object.get_label_objects(0)

In [6]:
track = Track(maxObject=50, threshold=10)

In [7]:
def dep_fusion(label_object):
    margin = 10
    interval_size = 30
    dep = ((label_object.t[0]**2) + (label_object.t[1]**2) + (label_object.t[2]**2))**0.5
    dep_sig = 1/(1+np.exp(-dep/10))
    # degree
    thita = np.arccos(label_object.t[0]/((label_object.t[0])**2 + (label_object.t[2])**2)**0.5)
    n = int(thita/np.pi/6)
    dep_fusion = dep_sig*interval_size + n*(margin+interval_size)
    return dep_fusion

In [9]:
size = (1242, 375)
fourcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
output_path = './cat_devided.mp4'
fps = 12
writer = cv2.VideoWriter(output_path, fourcc, fps, size)
frame_id = 0
object_sum = 0
while frame_id < 150:
    frame_output = []
    num = len([x for x in label_object if x.frame_idx==frame_id])
    for i in range(object_sum, object_sum + num):
        entity = {}
#         entity['dep'] = ((label_object[i].t[0]**2) + (label_object[i].t[2]**2))**0.5
        entity['dep'] = dep_fusion(label_object[i])
        entity['center'] = label_object[i].t
        entity['dim'] = np.array([label_object[i].h, label_object[i].w, label_object[i].l])
        entity['ort'] = label_object[i].ry
        entity['cat'] = abs(label_object[i].type)
        frame_output.append(entity)
    track.step(frame_output)
    frame = kitti_object.get_image(0, frame_id)
    for trackers in track.trackers:
        boxes_3d = []
        for tracker in trackers.values():
            if tracker.life == 3:
                boxes_3d.append(compute_box_3d_tracker(tracker, calib_object.P))
        frame_trackers = list(trackers.values())
        for x in range(len(boxes_3d)):
            frame = draw_projected_box3d_label(frame, boxes_3d[x][0], frame_trackers[x])
#     img = Image.fromarray(cv2.cvtColor(img_with_boundingBox, cv2.COLOR_BGR2RGB))
#     img.show()
    frame_id += 1
    object_sum += num
    writer.write(frame)
writer.release()

In [10]:
frame_output = []
num = len([x for x in label_object if x.frame_idx==0])
for i in range(num):
    entity = {}
    entity['dep'] = label_object[i].t[2]
    entity['center'] = label_object[i].t
    entity['dim'] = np.array([label_object[i].h, label_object[i].w, label_object[i].l])
    entity['ort'] = label_object[i].ry
    frame_output.append(entity)

In [11]:
frame_output

[{'dep': 13.410495,
  'center': (-4.552284, 1.858523, 13.410495),
  'dim': array([2.      , 1.823255, 4.433886]),
  'ort': -2.115488},
 {'dep': 5.776261,
  'center': (1.6404, 1.67566, 5.776261),
  'dim': array([1.739063, 0.824591, 1.785241]),
  'ort': -1.675458},
 {'dep': 8.455685,
  'center': (6.301919, 1.652419, 8.455685),
  'dim': array([1.714062, 0.767881, 0.972283]),
  'ort': -1.900245}]

In [12]:
track.step(frame_output)

In [13]:
track.trackers

{13.410495: <Tracker.Tracker at 0x20b2d1f0908>,
 5.776261: <Tracker.Tracker at 0x20b2d1f0388>,
 8.455685: <Tracker.Tracker at 0x20b2d1f0108>}

In [14]:
boxes_3d = []
for tracker in track.trackers.values():
    boxes_3d.append(compute_box_3d_tracker(tracker, calib_object.P))
pre_frame = kitti_object.get_image(0, 0)
frame_trackers = list(track.trackers.values())
for x in range(len(boxes_3d)):
    img_with_boundingBox = draw_projected_box3d_label(pre_frame, boxes_3d[x][0], frame_trackers[x])
img = Image.fromarray(cv2.cvtColor(img_with_boundingBox, cv2.COLOR_BGR2RGB))
img.show()

In [15]:
now_output = []
num2 = len([x for x in label_object if x.frame_idx==1])
for i in range(num, num+num2):
    entity = {}
    entity['dep'] = label_object[i].t[2]
    entity['center'] = label_object[i].t
    entity['dim'] = np.array([label_object[i].h, label_object[i].w, label_object[i].l])
    entity['ort'] = label_object[i].ry
    now_output.append(entity)

In [16]:
now_output

[{'dep': 13.581085,
  'center': (-4.650955, 1.766774, 13.581085),
  'dim': array([2.      , 1.823255, 4.433886]),
  'ort': -2.121565},
 {'dep': 5.778596,
  'center': (1.70064, 1.640419, 5.778596),
  'dim': array([1.739063, 0.824591, 1.785241]),
  'ort': -1.664456},
 {'dep': 8.156156,
  'center': (6.352093, 1.593046, 8.156156),
  'dim': array([1.714062, 0.767881, 0.972283]),
  'ort': -1.88684}]

In [18]:
track.step(now_output)

In [19]:
track.trackers

{5.778596: <Tracker.Tracker at 0x168aae5fe08>,
 8.156156: <Tracker.Tracker at 0x168aae5f888>,
 13.581085: <Tracker.Tracker at 0x168aae5ffc8>}

In [17]:
print(tracker.trackers[5.778596])
print(tracker.trackers[8.156156])

id:46, dep:5.778596, dim:[1.739063,0.824591,1.785241]
id:42, dep:8.156156, dim:[1.714062,0.767881,0.972283]


In [23]:
boxes_3d = []
for tracker in track.trackers.values():
    boxes_3d.append(compute_box_3d_tracker(tracker, calib_object.P))
pre_frame = kitti_object.get_image(0, 0)
frame_trackers = list(track.trackers.values())
for x in range(len(boxes_3d)):
    img_with_boundingBox = draw_projected_box3d_label(pre_frame, boxes_3d[x][0], frame_trackers[x])
img = Image.fromarray(cv2.cvtColor(img_with_boundingBox, cv2.COLOR_BGR2RGB))
img.show()

In [25]:
now_output = []
num2 = len([x for x in label_object if x.frame_idx==2])
for i in range(num, num+num2):
    entity = {}
    entity['dep'] = label_object[i].t[2]
    entity['center'] = label_object[i].t
    entity['dim'] = np.array([label_object[i].h, label_object[i].w, label_object[i].l])
    entity['ort'] = label_object[i].ry
    now_output.append(entity)
track.step(now_output)
boxes_3d = []
for tracker in track.trackers.values():
    boxes_3d.append(compute_box_3d_tracker(tracker, calib_object.P))
pre_frame = kitti_object.get_image(0, 0)
frame_trackers = list(track.trackers.values())
for x in range(len(boxes_3d)):
    img_with_boundingBox = draw_projected_box3d_label(pre_frame, boxes_3d[x][0], frame_trackers[x])
img = Image.fromarray(cv2.cvtColor(img_with_boundingBox, cv2.COLOR_BGR2RGB))
img.show()